## Logistic Regression for Sentiment Analysis

Adapted from http://nbviewer.jupyter.org/github/rasbt/pattern_classification/blob/master/machine_learning/scikit-learn/outofcore_modelpersistence.ipynb

<br>
<br>

## The IMDb Movie Review Dataset

In this section, we will train a simple logistic regression model to classify movie reviews from the 50k IMDb review dataset that has been collected by Maas et. al.

> AL Maas, RE Daly, PT Pham, D Huang, AY Ng, and C Potts. Learning word vectors for sentiment analysis. In Proceedings of the 49th Annual Meeting of the Association for Computational Lin- guistics: Human Language Technologies, pages 142–150, Portland, Oregon, USA, June 2011. Association for Computational Linguistics

[Source: http://ai.stanford.edu/~amaas/data/sentiment/]

The dataset consists of 50,000 movie reviews from the original "train" and "test" subdirectories. The class labels are binary (1=positive and 0=negative) and contain 25,000 positive and 25,000 negative movie reviews, respectively.
For simplicity, I assembled the reviews in a single CSV file.


In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
# if you want to download the original file:
#df = pd.read_csv('https://raw.githubusercontent.com/rasbt/pattern_classification/master/data/50k_imdb_movie_reviews.csv')
# otherwise load local file
df = pd.read_csv('shuffled_movie_data.csv')
df.tail()

,review,sentiment
49995,"OK, lets start with the best. the building. al...",0
49996,The British 'heritage film' industry is out of...,0
49997,I don't even know where to begin on this one. ...,0
49998,Richard Tyler is a little boy who is scared of...,0
49999,I waited long to watch this movie. Also becaus...,1


Let us shuffle the class labels.

In [2]:
import numpy as np
## uncomment these lines if you have dowloaded the original file:
#np.random.seed(0)
#df = df.reindex(np.random.permutation(df.index))
#df[['review', 'sentiment']].to_csv('shuffled_movie_data.csv', index=False)

<br>
<br>

## Preprocessing Text Data

Now, let us define a simple `tokenizer` that splits the text into individual word tokens. Furthermore, we will use some simple regular expression to remove HTML markup and all non-letter characters but "emoticons," convert the text to lower case, remove stopwords, and apply the Porter stemming algorithm to convert the words into their root form.

In [3]:
import numpy as np
from nltk.stem.porter import PorterStemmer
import re
from nltk.corpus import stopwords

stop = stopwords.words('english')
porter = PorterStemmer()

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    text = [w for w in text.split() if w not in stop]
    tokenized = [porter.stem(w) for w in text]
    return text

Let's give it at try:

In [4]:
tokenizer('This :) is a <a> test! :-)</br>')

['test', ':)', ':)']

## Learning (SciKit)

First, we define a generator that returns the document body and the corresponding class label:

In [5]:
def stream_docs(path):
    with open(path, 'r') as csv:
        next(csv) # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

To conform that the `stream_docs` function fetches the documents as intended, let us execute the following code snippet before we implement the `get_minibatch` function:

In [6]:
next(stream_docs(path='shuffled_movie_data.csv'))

('"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />""Murder in Greenwich"" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich f

After we confirmed that our `stream_docs` functions works, we will now implement a `get_minibatch` function to fetch a specified number (`size`) of documents:

In [7]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    for _ in range(size):
        text, label = next(doc_stream)
        docs.append(text)
        y.append(label)
    return docs, y

Next, we will make use of the "hashing trick" through scikit-learns [HashingVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.HashingVectorizer.html) to create a bag-of-words model of our documents. Details of the bag-of-words model for document classification can be found at  [Naive Bayes and Text Classification I - Introduction and Theory](http://arxiv.org/abs/1410.5329).

In [8]:
from sklearn.feature_extraction.text import HashingVectorizer
vect = HashingVectorizer(decode_error='ignore', 
                         n_features=2**21,
                         preprocessor=None, 
                         tokenizer=tokenizer)

# Exercise 1: define features based on word embeddings (pre-trained word2vec / Glove/Fastext emebddings can be used)
# Define suitable d dimension, and sequence length

Using the [SGDClassifier]() from scikit-learn, we will can instanciate a logistic regression classifier that learns from the documents incrementally using stochastic gradient descent. 

In [9]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss='log', random_state=1, n_iter=1)
doc_stream = stream_docs(path='shuffled_movie_data.csv')
# Exercise 2: Define at least a Three layer neural network. Define its structure (number of hidden neurons, etc)
# Define a nonlinear function for hidden layers.
# Define a suitable loss function for binary classification
# Implement the backpropagation algorithm for this structure
# Do not use Keras / Tensorflow /PyTorch etc. libraries
# Train the model using SGD

In [10]:
#import pyprind
#pbar = pyprind.ProgBar(45)

classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    X_train = vect.transform(X_train)
#     print(y_train.shape)
    clf.partial_fit(X_train, y_train, classes=classes)
    #pbar.update()

/home/david/entorno1/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:130: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
/home/david/entorno1/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:130: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
/home/david/entorno1/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:130: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
/home/david/entorno1/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:130: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
/home/david/entorno1/lib/python3.6/site-packages

/home/david/entorno1/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:130: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
/home/david/entorno1/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:130: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
/home/david/entorno1/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:130: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
/home/david/entorno1/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:130: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
/home/david/entorno1/lib/python3.6/site-packages

Depending on your machine, it will take about 2-3 minutes to stream the documents and learn the weights for the logistic regression model to classify "new" movie reviews. Executing the preceding code, we used the first 45,000 movie reviews to train the classifier, which means that we have 5,000 reviews left for testing:

In [11]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print('Accuracy: %.3f' % clf.score(X_test, y_test))
#Exercise 3: compare  with your Neural Network

Accuracy: 0.867


I think that the predictive performance, an accuracy of ~87%, is quite "reasonable" given that we "only" used the default parameters and didn't do any hyperparameter optimization. 

After we estimated the model perfomance, let us use those last 5,000 test samples to update our model.

In [12]:
clf = clf.partial_fit(X_test, y_test)

/home/david/entorno1/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:130: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


<br>
<br>

# Model Persistence

In the previous section, we successfully trained a model to predict the sentiment of a movie review. Unfortunately, if we'd close this IPython notebook at this point, we'd have to go through the whole learning process again and again if we'd want to make a prediction on "new data."

So, to reuse this model, we could use the [`pickle`](https://docs.python.org/3.5/library/pickle.html) module to "serialize a Python object structure". Or even better, we could use the [`joblib`](https://pypi.python.org/pypi/joblib) library, which handles large NumPy arrays more efficiently.

To install:
conda install -c anaconda joblib

In [13]:
import joblib
import os
if not os.path.exists('./pkl_objects'):
    os.mkdir('./pkl_objects')
    
joblib.dump(vect, './vectorizer.pkl')
joblib.dump(clf, './clf.pkl')

['./clf.pkl']

Using the code above, we "pickled" the `HashingVectorizer` and the `SGDClassifier` so that we can re-use those objects later. However, `pickle` and `joblib` have a known issue with `pickling` objects or functions from a `__main__` block and we'd get an `AttributeError: Can't get attribute [x] on <module '__main__'>` if we'd unpickle it later. Thus, to pickle the `tokenizer` function, we can write it to a file and import it to get the `namespace` "right".

In [14]:
%%writefile tokenizer.py
from nltk.stem.porter import PorterStemmer
import re
from nltk.corpus import stopwords

stop = stopwords.words('english')
porter = PorterStemmer()

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    text = [w for w in text.split() if w not in stop]
    tokenized = [porter.stem(w) for w in text]
    return text

Writing tokenizer.py


In [15]:
from tokenizer import tokenizer
joblib.dump(tokenizer, './tokenizer.pkl')

['./tokenizer.pkl']

Now, let us restart this IPython notebook and check if the we can load our serialized objects:

In [16]:
import joblib
tokenizer = joblib.load('./tokenizer.pkl')
vect = joblib.load('./vectorizer.pkl')
clf = joblib.load('./clf.pkl')

After loading the `tokenizer`, `HashingVectorizer`, and the tranined logistic regression model, we can use it to make predictions on new data, which can be useful, for example, if we'd want to embed our classifier into a web application -- a topic for another IPython notebook.

In [17]:
example = ['I did not like this movie']
X = vect.transform(example)
clf.predict(X)

array([0])

In [18]:
example = ['I loved this movie']
X = vect.transform(example)
clf.predict(X)

array([1])

# Solucion de Tareas

## Ejercicio 1: WordEmbeddings

Para este ejercicio se utiliza word Embeddigs preentrenados Glove. Primeramente se carga el dataset preentrenado, a continuacion se define un vocabulario del data set original (movie reviews) y finalmente se convierten los documentos a vectores numericos.

## Cargando dataset

In [19]:
#RAW DATA
doc_stream = stream_docs(path='shuffled_movie_data.csv')
TOTAL_EXAMPLES = 50000

X_full , y_full = get_minibatch(doc_stream, size=TOTAL_EXAMPLES)

## Definiendo nuevas caracteristicas usando Word Embeddings

In [20]:
#configuracion
MAX_NB_WORDS = 1000
MAX_SEQUENCE_LENGTH = 1000
N_FEATURES = 50 #dimension


In [21]:
#Carga de word Embedings preentrenados
glove_file = 'pretrained/glove.twitter.27B.' + str(N_FEATURES) + 'd.txt'
emb_dict = {}
glove = open(glove_file)
for line in glove:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:], dtype='float32')
    emb_dict[word] = vector
glove.close()

In [22]:

#Creacion del vocabulario del dataset

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

#MAX_NB_WORDS = 1000
#MAX_SEQUENCE_LENGTH = 1000
#N_FEATURES = 500 #dimensiones

tokenizer_k = Tokenizer(nb_words=MAX_NB_WORDS,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
               lower=True,split=" ")
tokenizer_k.fit_on_texts(X_full)
sequences = tokenizer_k.texts_to_sequences(X_full)
word_index = tokenizer_k.word_index #vocabulario del dataset
N_WORDS = len(word_index) #numero de palabras unicas en el dataset
print('%s palabras unicas.' %N_WORDS)

Using TensorFlow backend.


124252 palabras unicas.


## Vectorizacion del Dataset

In [23]:
# Funcion que convierte un documento en un vector promediando vectores "wordEmbeddings" 
# en un documento
def document2Vector(vocabulario_emb,n_features,documento):
    # n_features: dimension del vector
    # documentos: documento (string)
    vector = np.zeros((1,n_features),dtype="float64")
    words = tokenizer(documento)
    nwords = 0
    for word in words:
        #print(word)
        word_emb = vocabulario_emb.get(word)
        if word_emb is not None:
            vector = vector+word_emb
            nwords = nwords + 1.
    if nwords > 0:
        vector = vector/nwords
    vector = vector.reshape((1,n_features))
    
    return vector

In [24]:
#funcion que transforma documentos(corpus) a vectores numericos usando word embeddings.
def corpus2Matrix(corpus,n_examples, n_features):
    matrix = np.zeros((n_examples, n_features))
    for i in range(n_examples):
        vector = document2Vector(emb_dict,n_features,corpus[i])
        matrix[i,:]=vector
    return matrix

## Definicion de nuevas Caracteristicas
Se procede a mapear los word Embedding a los documentos. Como los documentos son un conjunto de palabras, se procedio a calcular el promedio de los word Embeddings que componen un documento para obtener un vector que represente al documento(movie review). Este proceso puede demorar de 4 a 5 minutos por lo que se realizo una sola vez y se guardo el resultado en un csv.

In [25]:
#DEFINICION DE NUEVAS CARACTERISTICAS: Llamado de las funciones antes definidas.
#Este proceso puede demorar, por lo que solo se ejecuto una vez y se guardo la matriz en
#un csv, si desea ejecutar descomente las siguientes 2 lineas
#mm=corpus2Matrix(X_full,TOTAL_EXAMPLES, N_FEATURES)
#np.savetxt('matrix_embeding.csv', mm)

In [26]:
# Carga de datos
X=np.loadtxt('matrix_embeding.csv')

In [27]:
X[4999,:]*1000

array([ 2.32962105e+02,  2.10725443e+02,  1.55017363e+01,  8.40763092e+01,
        7.92159411e+01,  2.86586536e+02,  9.04910358e+02, -2.36633981e+02,
       -1.82299615e+02,  1.61963325e+02, -4.05243644e+01, -1.24028262e+02,
       -3.84192170e+03, -2.06049839e+01, -2.16941698e+02,  2.11767037e+01,
        2.13507740e+02, -1.96449158e+02, -1.55626545e+02, -8.81555390e+01,
        7.58261087e+01, -3.86509834e+01, -7.65415855e+01,  1.90189775e+02,
       -1.42897729e+02,  6.17523400e+02, -1.26982207e+02,  8.89971795e+01,
       -1.81705949e+02, -1.76245184e+02, -7.33593443e-01, -9.08780513e+01,
       -5.60791226e+01,  3.49233080e+01,  2.46382283e+02,  9.17318119e+01,
       -1.31053947e+02, -1.84546897e+02,  2.04173912e+02, -1.01386669e+02,
       -3.25451896e+02,  2.00178957e+02,  2.67673428e+02, -3.65778283e+01,
        9.00153811e+01, -1.20048910e+02,  2.05044827e+02,  2.73416869e+02,
       -2.22464404e+02,  1.64775535e+01])

# Ejercicio 2: Implementacion de Red Neuronal

## Division de datos de entrenamiento, validacion y test

In [28]:
from sklearn.model_selection import train_test_split

m_train = 45000 #numero de ejemplos de entrenamiento
m_test = 5000 #numero de ejemplos de prueba
y_full = np.array(y_full)
y_full = y_full.reshape(TOTAL_EXAMPLES,1)
X_train, X_test, y_train, y_test = train_test_split(X, y_full, test_size=0.1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)



## Entrenamiento del modelo usando sklearn
En esta parte se entreno un modelo de SKlearn con las nuevas caracteristicas definidas, la sxactitud que se obtuvo coneste modelo es de 78.1%

In [29]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(alpha=0.001,loss='log', random_state=1, n_iter=15)
clf.fit(X_train, y_train)

/home/david/entorno1/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:130: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


SGDClassifier(alpha=0.001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=15, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=1, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [30]:
print('Accuracy: %.3f' % clf.score(X_test, y_test))

Accuracy: 0.781


In [31]:
X_train = np.transpose(X_train)
y_train = np.transpose(y_train)
X_test = np.transpose(X_test)
y_test = np.transpose(y_test)
X_val = np.transpose(X_val)
y_val = np.transpose(y_val)
print('ejemplos entrenamiento: ',X_train.shape)
print('ejemplos entrenamiento (labels): ',y_train.shape)
print('ejemplos validacion: ',X_val.shape)
print('ejemplos validacion (labels): ',y_val.shape)
print('ejemplos test: ',X_test.shape)
print('ejemplos test (labels): ',y_test.shape)

ejemplos entrenamiento:  (50, 36000)
ejemplos entrenamiento (labels):  (1, 36000)
ejemplos validacion:  (50, 9000)
ejemplos validacion (labels):  (1, 9000)
ejemplos test:  (50, 5000)
ejemplos test (labels):  (1, 5000)


## Definicion de la Arquitectura de la Red Neuronal

En las siguientes secciones se definen funciones para una red neuronal de 3 capas. La primera capa consta de 50 neuronas (Caracteristicas Word Embeddings), la capa oculta con 4 neuronas (con la tangente hyperbolica como funcion de activacion) y la capa de salida con una neurona cuya funcion de activacion es la sigmoide.

In [32]:
# Funcion que define la arquitectura de la red neuronal
def layer_sizes(X, Y, n_h):
    """
    X -- ejemplos de entrenamiento
    Y -- etiquetas de X
    
    Returns:
    n_x -- numero de neuronas en la capa de entrada
    n_h -- numero de neuronas de la capa oculta
    n_y -- numero de neuronas de la capa de salida
    """
    n_x = X.shape[0] # input layer
    n_h = n_h
    n_y = Y.shape[0] # output layer
    
    return (n_x, n_h, n_y)

In [33]:
(n_x, n_h, n_y) = layer_sizes(X_train, y_train, 4)
print("Tamaño de capa de entrada: n_x = " + str(n_x))
print("Tamaño de capa  oculta: n_h = " + str(n_h))
print("Tamaño de capa de salida: n_y = " + str(n_y))

Tamaño de capa de entrada: n_x = 50
Tamaño de capa  oculta: n_h = 4
Tamaño de capa de salida: n_y = 1


## Inicializacion de parametros

In [34]:
def initialize_parameters(n_x, n_h, n_y):
    """
    n_x -- Tamaño de capa de entrada
    n_h -- Tamaño de capa  oculta
    n_y -- Tamaño de capa de salida
    
    Returns:
    params -- diccionario con parametros
    """
    
    W1 = np.random.randn(n_h,n_x)*0.01
    b1 = np.zeros((n_h,1))
    W2 = np.random.randn(n_y,n_h)*0.01
    b2 = np.zeros((n_y,1))
    
    assert (W1.shape == (n_h, n_x))
    assert (b1.shape == (n_h, 1))
    assert (W2.shape == (n_y, n_h))
    assert (b2.shape == (n_y, 1))
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [35]:
parameters = initialize_parameters(n_x, n_h, n_y)
print("W1 = " + str(parameters["W1"].shape))
print("b1 = " + str(parameters["b1"].shape))
print("W2 = " + str(parameters["W2"].shape))
print("b2 = " + str(parameters["b2"].shape))

W1 = (4, 50)
b1 = (4, 1)
W2 = (1, 4)
b2 = (1, 1)


## Funciones de activacion

In [36]:
#definicion de funcion sigmoide
def sigmoid(z):
    s=1/(1+np.exp(-z))    
    return s

#Funcion tangente hyperbolica
def s_tanh(z):
    t = np.tanh(z)
    return t

## Forward Propagation

In [37]:
def s_forward_propagation(X, parameters):
    """
    X -- datos de entrenamiento(n_x, m)
    parameters -- diccionario con parametros inicializados
    
    Returns:
    A2 -- Activacion de la capa final (sigmoide)
    cache -- Diccionario con calculos intermedios "Z1", "A1", "Z2" y "A2"
    """
    # Parametros
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    
    Z1 = np.dot(W1,X)+b1
#     A1 = sigmoid(Z1)
    A1 = np.tanh(Z1)
    Z2 = np.dot(W2,A1)+b2
    A2 = sigmoid(Z2)
    
    assert(A2.shape == (1, X.shape[1]))
    
    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    
    return A2, cache

## Funcion de costo

In [38]:
def s_compute_cost(a2, yi, parameters,lambd,m):
    """
    a2 -- Salida de activacion de la ultima capa para un ejemplo
    yi -- etiqueta del ejemplo
    parameters -- diccionario con parametros W1, b1, W2 and b2
    lamb -- parametro de regularizacion
    m -- numero de ejemplos de entrenamiento
    
    Returns:
    cost -- funcion de costo (cross-entropy)
    """
    
    a2 = np.squeeze(a2)
    yi = np.squeeze(yi)
    
    W1 = parameters["W1"]
    W2 = parameters["W2"]
    L2_regularizacion = (lambd/(2*m))*(np.sum(np.square(W2))+np.sum(np.square(W1)))
    
    logprobs = yi*np.log(a2) + (1-yi)*np.log(1-a2)
    cost = -logprobs + L2_regularizacion
    
    cost = np.squeeze(cost)
#     print('costo: ',cost)
    assert(isinstance(cost, float))
    
    return cost

In [39]:
# Funcion que se encarga de actualizar los parametros despues de una epoca

def s_update_parameters(parameters, grads, learning_rate):
    """
    parameters -- parametros
    grads -- gradientes
    learning_rate -- coeficiente de aprendizaje
    
    Returns:
    parameters -- parametros actualizados
    """
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    
    dW1 = grads["dW1"]
    db1 = grads["db1"]
    dW2 = grads["dW2"]
    db2 = grads["db2"]
    
    # Actualizacion
    W1 = W1-learning_rate*dW1
    b1 = b1-learning_rate*db1
    W2 = W2-learning_rate*dW2
    b2 = b2-learning_rate*db2
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

## Backward Propagation

In [40]:
def s_backward_propagation(parameters, cache, X, Y, i,lambd):
    
    m = X.shape[1]
    n = X.shape[0]
    
    # Activaciones de forward prop.
    A1 = cache["A1"]
    A2 = cache["A2"]
   
    #indice aleatorio
#     i = np.random.random_integers(0, m-1) 
#     print('i',i)
   
    
    grads = {"dW1": None,
             "db1": None,
             "dW2": None,
             "db2": None}
      
    W1 = parameters["W1"]
    W2 = parameters["W2"]
        
    a1 = A1[:,i].reshape(A1[:,i].shape[0],1) #activacion de un ejemplo en la primera capa
    a2 = A2[:,i].reshape(A2[:,i].shape[0],1) #activacion de un ejemplo en la ultima capa

    xi = X[:,i].reshape(X[:,i].shape[0],1)#ejemplo aleatorio
    yi = Y[:,i].reshape(Y[:,i].shape[0],1)#etiqueta

        ##Calculo del costo para un unico ejemplo
    costo = s_compute_cost(a2, yi, parameters,lambd,m)
#         costo = compute_cost(A2, Y, parameters)
        
        ## Backward propagation: calculo de dW1, db1, dW2, db2. 
    dZ2 = a2 - yi    
    grads["dW2"] = dZ2*np.transpose(a1) + (lambd/m)*(W2)
    grads["db2"] = dZ2#1x1
#     gp = sigmoid(a1)*(1-sigmoid(a1))#derivada de sigmoide
    gp = 1-np.power(a1,2)#derivada de tanh
    dZ1 = np.multiply(np.dot(np.transpose(W2),dZ2),gp)
    grads["dW1"] = np.dot(dZ1,np.transpose(xi))+ (lambd/m)*(W1)
    grads["db1"] = dZ1
        ##update
    
#     grads = {"dW1": dW1,
#              "db1": db1,
#              "dW2": dW2,
#              "db2": db2}
    
    return  costo, grads

## Gradiente Descendente Estocastico

In [49]:
def nn_modelo(X, Y, n_h, num_iterations, print_cost,learning_rate,lambd):
    """
    Argumentos:
    X -- conjunto de entrenamiento
    Y -- etiquetas de entrenamiento
    n_h -- numero de neuronas en la capa oculta
    num_iterations -- Numero de epocas 
    print_cost -- imprimir costos
    
    Retorna:
    parameters -- parametros aprendidos.
    costos -- array historico de costos
    """
    
    n_x, n_h, n_y = layer_sizes(X, Y,n_h)
    print('(n_x, n_h, n_y):',n_x, n_h, n_y)
#     n_h = layer_sizes(X, Y)[1]
#     n_y = layer_sizes(X, Y)[2]
    
   ##Inicializacion de parametros
    parameters = initialize_parameters(n_x, n_h, n_y)
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    
    ph_W1 = []
    ph_b1 = []
    ph_W2 = []
    ph_b2 = []
    
    # Gradiente Descendente Estocatico
    costos = []
    m = X.shape[1]
        
    for j in range(0, num_iterations):
#         i = np.random.random_integers(0, m-1)
#         # Forward propagation.
#         A2, cache = s_forward_propagation(X, parameters)
#         print('iteracion: ',j)
#         # Backpropagation. 
#         costo, grads = s_backward_propagation(parameters, cache, X, Y,i)
#         costos.append(costo)
#          # Gradient descent parameter update. 
#         parameters = s_update_parameters(parameters, grads,learning_rate)
#         if(costo<0.01):
#             break
        for i in range(0,m):
            ph_W1.append(parameters["W1"])
            ph_b1.append(parameters["b1"])
            ph_W2.append(parameters["W2"])
            ph_b2.append(parameters["b2"])
            
            # Forward propagation.
            A2, cache = s_forward_propagation(X, parameters)
#             print('iteracion: ',i)
            # Backpropagation. 
            costo, grads = s_backward_propagation(parameters, cache, X, Y,i,lambd)
            costos.append(costo)
            # Gradient descent parameter update. 
            
            parameters = s_update_parameters(parameters, grads,learning_rate)
            if(costo<0.01):
                break
    
   
    parameters_h = {"hW1": ph_W1,
             "hb1": ph_b1,
             "hW2": ph_W2,
             "hb2": ph_b2}
    
    parameters = calculate_mean_parameters(parameters,parameters_h)
    
   
    return costos,parameters

Debido a que el algoritmo SGD calcula costos que oscilan entre el minimo global, se promedia los parametros para una mejor aproximacion. (Fuente: https://www.coursera.org/learn/ml-classification/lecture/iDOIY/don-t-trust-last-coefficients)

In [42]:
#Funcion que calcula la media de todos los parametros calculados

def calculate_mean_parameters(parameters, parameters_history):
    ph_W1 = parameters_history["hW1"]
    ph_b1 = parameters_history["hb1"]
    ph_W2 = parameters_history["hW2"]
    ph_b2 = parameters_history["hb2"]
    
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    
    length = len(ph_W1)
    for k in range(0,length-1):
        W1 = W1 + ph_W1[k]
        b1 = b1 + ph_b1[k]
        W2 = W2 + ph_W2[k]
        b2 = b2 + ph_b2[k]
        
    W1 = (1/length)*W1
    b1 = (1/length)*b1
    W2 = (1/length)*W2
    b2 = (1/length)*b2
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    return parameters

In [43]:
## Funcion que realiza predicciones
## Threshold = 0.5
def s_predict(parameters, X):
    A2, cache = s_forward_propagation(X,parameters)
    predictions = (A2 > 0.5).astype(int)
    
    return predictions

In [44]:
##Funcion para graficar la convergencia de un algoritmo
import matplotlib.pyplot as plt
def plotConvergence(jvec,iterations):
    plt.figure(figsize=(10,6))
    plt.plot(range(len(jvec)),jvec,'bo')
    plt.grid(True)
    plt.title("Convergencia Stochastic Gradient Descendt")
    plt.xlabel("Numero de Iteraciones")
    plt.ylabel("Funcion de Costo")
    dummy = plt.xlim([-0.05*iterations,1.05*iterations])
#     dummy = plt.xlim([-0.05*iterations,10.0*iterations])

## Entrenamiento

In [50]:
costos, parameters = nn_modelo(X_train, y_train, 4, num_iterations=1, 
                               print_cost=True,learning_rate=0.01, lambd = 0.05)


(n_x, n_h, n_y): 50 4 1


In [54]:
# costos
# len(costos)
# plotConvergence(costos,len(costos))
# length = 500
# plotConvergence(costos[len(costos)-length:len(costos)],length)

In [51]:
predictions = s_predict(parameters, X_test)
accuracy = float((np.dot(y_test,predictions.T) + np.dot(1-y_test,1-predictions.T))/float(y_test.size)*100)
print('Exactitud: ',accuracy)


Exactitud:  77.58


In [52]:
np.mean(predictions == y_test)

0.7758

## Ejercicio3: Comparacion de resultados

Comparando con el modelo de SKlearn con las nuevas caracteristicas definidas, la exactitud que se obtuvo con la red neuronal es de 77.6% mientras que con SKLearn es de 78.1%